In [119]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, lpSum, lpDot, value, LpBinary

In [120]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)


Current Working Directory: C:\Users\somad\OneDrive\Desktop\thesis


In [121]:

# Change the working directory
new_directory = "C:/Users/somad/OneDrive/Desktop/thesis"
os.chdir(new_directory)

# Verify the change
updated_directory = os.getcwd()
print("Updated Working Directory:", updated_directory)


Updated Working Directory: C:\Users\somad\OneDrive\Desktop\thesis


In [122]:
from pulp import LpProblem, LpVariable, lpSum, value, LpBinary,  LpMinimize
from geopy.distance import geodesic  # You may need to install the geopy library (pip install geopy)

In [123]:
# Read data from the Excel sheet
df = pd.read_excel('Customers.xlsx')  # Replace 'your_excel_file.xlsx' with your file name

In [124]:
df

,Name,Latitude_(y),Longitude_(x),Federal State,Factory Location,Cust_Category,Daily_Demand,Fixed_cost
0,AKRO_PLASTIC_GmbH_Niederzissen,50.4410,7.2300,Rheinlad-Pfalz,Niederzissen,Low,1.626016,0.050000
1,B_Braun_Bad_Arolsen,51.3794,8.9998,Hesse,Bad Arolsen,Mid,3.252033,0.084728
2,B_Braun_Melsungen_Schwarzenbergerweg77,51.1397,9.5456,Hesse,Melsungen,Mid,3.252033,0.084728
3,B_Braun_Melsungen_Schwarzenbergerweg21,51.1372,9.5435,Hesse,Melsungen,Mid,3.252033,0.084728
4,B_Braun_Melsungen_Carlbraunstr,51.1312,9.5518,Hesse,Melsungen,Mid,3.252033,0.084728
...,...,...,...,...,...,...,...,...
118,Volkswagen_Zuffenhausen,48.8358,9.1525,Baden-Württemberg,Zuffenhausen,High,4.878049,0.150125
119,Volkswagen_Zwikau,50.7936,12.4875,Saxony,Zwikau,High,4.878049,0.037672
120,Westlake_Vinnolite_Burgkirchen_an_der_Alz,48.1781,12.7228,Bavaria,Burgkirchen an der Alz,Mid,3.252033,0.185273
121,Wirthwein_Brandenburg_GmbH_&_Co._KG_Havel,52.3863,12.4261,Brandenburg,Havel,Low,1.626016,0.022029


In [125]:
# Function to calculate Euclidean distance between two locations
def euclidean_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

# Set up the PuLP problem
prob = LpProblem("Facility_Location_Model", LpMinimize)

# Decision variables
y = {j: LpVariable(f"y_{j}", 0, 1, LpBinary) for j in df.index}
x = {(i, j): LpVariable(f"x_{i}_{j}", 0, 1, LpBinary) for i in df.index for j in df.index}


In [126]:
# Objective function
fixed_costs = lpSum(df.at[j, 'Fixed_cost'] * y[j] for j in df.index)
variable_costs = lpSum(df.at[i, 'Daily_Demand'] * euclidean_distance(df.at[i, 'Latitude_(y)'], df.at[i, 'Longitude_(x)'],
                                                                     df.at[j, 'Latitude_(y)'], df.at[j, 'Longitude_(x)']) * x[i, j]
                      for i in df.index for j in df.index)
prob += fixed_costs + variable_costs

In [127]:
# Constraints
# (1) Open at most 3 stores
prob += lpSum(y[j] for j in df.index) <= 4

# (2) Customers from city 𝑖 can only be assigned to city 𝑗 if a store is opened in city 𝑗
for i in df.index:
    for j in df.index:
        prob += x[i, j] <= y[j]

# (3) Customers from each city are assigned to exactly one store
for i in df.index:
    prob += lpSum(x[i, j] for j in df.index) == 1

In [128]:
# Solve the problem
prob.solve()

# Print the results
print("Objective Value:", prob.objective.value())
print("\nStore Locations:")
for j in df.index:
    print(f"{df.at[j, 'Name']}: {y[j].value()}")

Objective Value: 399.11706636673955

Store Locations:
AKRO_PLASTIC_GmbH_Niederzissen: 0.0
B_Braun_Bad_Arolsen: 0.0
B_Braun_Melsungen_Schwarzenbergerweg77: 0.0
B_Braun_Melsungen_Schwarzenbergerweg21: 0.0
B_Braun_Melsungen_Carlbraunstr: 0.0
BASF_Ludwigshafen_Gartnereistrase: 0.0
BASF_Ludwigshafen_Rheinuferstraße: 0.0
BASF_Ludwigshafen_AmThein: 0.0
Beiersdorf_GmbH_Berlin: 0.0
Beiersdorf_Manufacturing_Hamburg_Hamburg: 0.0
BMW_Group_Berlin: 0.0
BMW_Group_Dingolfing: 0.0
BMW_Group_Eisenach: 0.0
BOSCH_Feuerbach: 0.0
BOSCH_Leinfelden_Echterdingen: 0.0
BOSCH_Stuttgart_Pragstraße: 0.0
BOSCH_Stuttgart_Heilbronnerstr: 0.0
BOSCH_Stuttgart_Kruppstraße: 0.0
Brose_GmbH_Berlin: 0.0
BSN_medical_GmbH_Hamburg: 1.0
Casar_Drahtseilwerk_Saar_GmbH_Kirkel: 0.0
Coca_Cola_Berlin: 0.0
Coca_Cola_Bielefeld: 0.0
Coca_Cola_Cologne: 0.0
Coca_Cola_Dorsten: 0.0
Coca_Cola_Erlangen: 0.0
Coca_Cola_Fürstenfeldbruck: 0.0
Coca_Cola_Gemmrigheim: 0.0
Coca_Cola_Herten: 0.0
Coca_Cola_Hohenschönhausen: 0.0
Coca_Cola_Karlsruhe: 0.0

In [129]:
# Print the selected store locations
print("\nSelected Store Locations:")
for j in df.index:
    if y[j].value() == 1.0:
        print(df.at[j, 'Name'])



Selected Store Locations:
BSN_medical_GmbH_Hamburg
Müller_Group__Düsseldorf
Volkswagen_Neckarsulm
Volkswagen_Zwikau


In [130]:
# Print the selected store locations with latitude and longitude
print("\nSelected Store Locations:")
for j in df.index:
    if y[j].value() == 1.0:
        name = df.at[j, 'Name']
        latitude = df.at[j, 'Latitude_(y)']
        longitude = df.at[j, 'Longitude_(x)']
        print(f"{name}: Latitude {latitude}, Longitude {longitude}")



Selected Store Locations:
BSN_medical_GmbH_Hamburg: Latitude 53.4762, Longitude 9.9069
Müller_Group__Düsseldorf: Latitude 51.1674, Longitude 6.843
Volkswagen_Neckarsulm: Latitude 49.2014, Longitude 9.2219
Volkswagen_Zwikau: Latitude 50.7936, Longitude 12.4875


In [131]:
# Solve the problem
prob.solve()

# Print the results
print("Objective Value:", prob.objective.value())
print("\nStore Locations:")
for j in df.index:
    if y[j].value() == 1.0:
        print(f"{df.at[j, 'Name']} ({df.at[j, 'Latitude_(y)']}, {df.at[j, 'Longitude_(x)']}):")
        for i in df.index:
            if x[i, j].value() == 1.0:
                print(f"   {df.at[i, 'Name']}")



Objective Value: 399.11706636673955

Store Locations:
BSN_medical_GmbH_Hamburg (53.4762, 9.9069):
   Beiersdorf_Manufacturing_Hamburg_Hamburg
   BSN_medical_GmbH_Hamburg
   Continental_Hannover
   Covestro_Bomlitz
   Covestro_Brunsbüttel
   Custom_Cells_Itzehoe_GmbH_Itzehoe
   Dräger_Lübeck
   Eppendorf_Polymere_GmbH_Wismar
   Eppendorf_SE_Hamburg
   Harry_Brot_Gmbh_Schenefeld
   Henkel_Hannover
   Henkel_Norderstedt
   PlantaCorp_GmbH_Hamburg
   Thyssenkrupp_Harz
   Unilever_Buxtehude
Müller_Group__Düsseldorf (51.1674, 6.843):
   AKRO_PLASTIC_GmbH_Niederzissen
   B_Braun_Bad_Arolsen
   Casar_Drahtseilwerk_Saar_GmbH_Kirkel
   Coca_Cola_Bielefeld
   Coca_Cola_Cologne
   Coca_Cola_Dorsten
   Coca_Cola_Herten
   Coca_Cola_Mönchengladbach
   Continental_Aachen
   Continental_Korbach
   Covestro_Krefeld_Uerdingen
   Covestro_Leverkusen
   Covestro_Meppen
   Eaton_Industries_Bad_Ems
   Evonik_Superabsorber_GmbH_Krefeld
   Henkel_Cologne
   Henkel_Krefeld
   MEDICE_Arzneimittel_Pütter_GmbH_&_

In [132]:
pip install plotly

In [133]:
!pip install plotly.express

In [134]:
import plotly.express as px

In [135]:
import pandas as pd
import plotly.express as px

# Your DataFrame with all 123 locations
df1 = pd.read_excel('Customers.xlsx')  # Replace 'your_file.xlsx' with your actual file path

# Specify colors for each Cust_Category
color_discrete_map = {'Low': 'green', 'Mid': 'orange', 'High': 'red'}

# Illustrate input data via Plotly Express
fig = px.scatter_mapbox(
    df1,
    lat='Latitude_(y)',
    lon='Longitude_(x)',
    color='Cust_Category',  # Color based on Cust_Category
    color_discrete_map=color_discrete_map,
    zoom=10,
    mapbox_style='open-street-map',
    width=1000,
    height=1000
)
fig.update_traces(marker={'size': 8})  # Adjust the size as needed
fig.show()




In [137]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Your DataFrame with all 123 locations
df1 = pd.read_excel('Customers.xlsx')  # Replace 'your_file.xlsx' with your actual file path

# Create a new column 'Facility' to label the facilities
df1['Facility'] = df1.apply(lambda row: 'Facility' if y[row.name].value() == 1.0 else 'Not Facility', axis=1)

# Illustrate the results via Plotly Express
fig = px.scatter_mapbox(
    df1,
    lat='Latitude_(y)',
    lon='Longitude_(x)',
    color='Facility',
    zoom=10,
    mapbox_style='open-street-map',
    width=1000,
    height=1000
)

# Add labels for facilities and their assigned customers
for j in df.index:
    if y[j].value() == 1.0:
        facility_name = df.at[j, 'Name']
        assigned_customers = [df.at[i, 'Name'] for i in df.index if x[i, j].value() == 1.0]
        facility_label = f"{facility_name}\nAssigned Customers: {', '.join(assigned_customers)}"
        fig.add_annotation(
            go.layout.Annotation(
                text=facility_label,
                x=df.at[j, 'Longitude_(x)'],
                y=df.at[j, 'Latitude_(y)'],
                showarrow=True,
                arrowhead=2,
                arrowcolor="black",
                arrowwidth=3,
                ax=0,
                ay=-40
            )
        )

fig.update_traces(marker={'size': 8})  # Adjust the size as needed
fig.show()


In [162]:
import folium
from IPython.display import display

# Create a map centered at the average latitude and longitude
center_lat = df['Latitude_(y)'].mean()
center_lon = df['Longitude_(x)'].mean()
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Create a map with custom width and height
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=6, width=1000, height=800)



## Add markers for facility locations
for j in df.index:
    if y[j].value() == 1.0:
        folium.Marker(
            location=[df.at[j, 'Latitude_(y)'], df.at[j, 'Longitude_(x)']],
            popup=df.at[j, 'Name'],
            icon=folium.Icon(color='blue')
        ).add_to(mymap)

# Add markers for customer locations and lines connecting customers to their assigned facility
for j in df.index:
    if y[j].value() == 1.0:
        for i in df.index:
            if x[i, j].value() == 1.0:
                folium.Marker(
                    location=[df.at[i, 'Latitude_(y)'], df.at[i, 'Longitude_(x)']],
                    popup=df.at[i, 'Name'],
                    icon=folium.Icon(color='green', icon_size=(20,20))
                ).add_to(mymap)
                folium.PolyLine(
                    locations=[[df.at[i, 'Latitude_(y)'], df.at[i, 'Longitude_(x)']],
                               [df.at[j, 'Latitude_(y)'], df.at[j, 'Longitude_(x)']]],
                    color='red'
                ).add_to(mymap)

# Display the map in the notebook
display(mymap)